Modify the helpers.py to load environment variables using python-dotenv:

In [10]:
# utils/helpers.py (Updated)

import os
import uuid
import json
import logging
from dotenv import load_dotenv
from datetime import datetime

# Load environment variables from .env
load_dotenv()

# Configure logging
logging.basicConfig(
    filename='chat_agent.log',
    level=logging.INFO,
    format='%(asctime)s:%(levelname)s:%(message)s'
)

def generate_session_token():
    """
    Generate a unique session token using UUID4.
    """
    return str(uuid.uuid4())

def load_knowledge_base():
    """
    Load knowledge base documents from the knowledge_base directory.
    Returns a dictionary with document names as keys and content as values.
    """
    knowledge_dir = os.path.join(os.getcwd(), 'knowledge_base')
    knowledge = {}
    for filename in os.listdir(knowledge_dir):
        if filename.endswith('.txt'):
            with open(os.path.join(knowledge_dir, filename), 'r', encoding='utf-8') as file:
                content = file.read()
                key = filename.replace('.txt', '')
                knowledge[key] = content
    return knowledge

def save_json(data, filename):
    """
    Save a dictionary as a JSON file.
    """
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

def load_json(filename):
    """
    Load a JSON file and return its content as a dictionary.
    """
    with open(filename, 'r', encoding='utf-8') as f:
        return json.load(f)

In [11]:
# utils/helpers.py

import os
import uuid
import json
import logging
from dotenv import load_dotenv
from datetime import datetime

# Load environment variables from .env
load_dotenv()

# Configure logging
logging.basicConfig(
    filename='chat_agent.log',
    level=logging.INFO,
    format='%(asctime)s:%(levelname)s:%(message)s'
)

def generate_session_token():
    """
    Generate a unique session token using UUID4.
    """
    return str(uuid.uuid4())

def load_knowledge_base():
    """
    Load knowledge base documents from the knowledge_base directory.
    Returns a dictionary with document names as keys and content as values.
    """
    knowledge_dir = os.path.join(os.getcwd(), 'knowledge_base')
    knowledge = {}
    for filename in os.listdir(knowledge_dir):
        if filename.endswith('.txt'):
            with open(os.path.join(knowledge_dir, filename), 'r', encoding='utf-8') as file:
                content = file.read()
                key = filename.replace('.txt', '')
                knowledge[key] = content
    return knowledge

def save_json(data, filename):
    """
    Save a dictionary as a JSON file.
    """
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

def load_json(filename):
    """
    Load a JSON file and return its content as a dictionary.
    """
    with open(filename, 'r', encoding='utf-8') as f:
        return json.load(f)

In [8]:
# storage/transcript_handler.py (Updated)

import json
import os
import gspread
from google.oauth2.service_account import Credentials
from utils.helpers import save_json, load_json, logging
import pandas as pd

# Load environment variables
GOOGLE_SHEETS_CREDENTIALS = os.getenv('GOOGLE_SHEETS_CREDENTIALS')
GOOGLE_SHEETS_SPREADSHEET_NAME = os.getenv('GOOGLE_SHEETS_SPREADSHEET_NAME')

# Authenticate and initialize Google Sheets client
def init_gspread():
    scopes = [
        "https://www.googleapis.com/auth/spreadsheets",
        "https://www.googleapis.com/auth/drive"
    ]
    credentials = Credentials.from_service_account_file(GOOGLE_SHEETS_CREDENTIALS, scopes=scopes)
    client = gspread.authorize(credentials)
    return client

def export_to_google_sheets(transcript_json):
    """
    Export the conversation transcript to Google Sheets.
    
    :param transcript_json: Dictionary containing the transcript data.
    """
    client = init_gspread()
    try:
        sheet = client.open(GOOGLE_SHEETS_SPREADSHEET_NAME).sheet1
    except Exception as e:
        logging.error(f"Error opening Google Sheet: {e}")
        return
    
    # Prepare data for export
    client_id = transcript_json.get('client_id', 'N/A')
    timestamp = transcript_json.get('timestamp', 'N/A')
    archetypes = transcript_json.get('archetypes', {})
    primary = archetypes.get('primary', 'N/A')
    secondary = archetypes.get('secondary', 'N/A')
    
    # Prepare questions and responses
    interview = transcript_json.get('interview', [])
    data = {
        'Client ID': client_id,
        'Timestamp': timestamp,
        'Primary Archetype': primary,
        'Secondary Archetype': secondary
    }
    
    for idx, qa in enumerate(interview, start=1):
        question = qa.get('question', '')
        response = qa.get('response', {}).get('answer', '')
        follow_up = qa.get('follow_up', '')
        example = qa.get('response', {}).get('example', '')
        
        data[f'Question {idx}'] = question
        data[f'Response {idx}'] = response
        data[f'Follow-Up {idx}'] = follow_up
        data[f'Example {idx}'] = example
    
    # Convert to DataFrame
    df = pd.DataFrame([data])
    
    # Append to Google Sheet
    try:
        sheet.append_rows(df.values.tolist())
        logging.info(f"Transcript exported to Google Sheets for Client ID: {client_id}")
    except Exception as e:
        logging.error(f"Error exporting to Google Sheets: {e}")

ImportError: cannot import name 'save_json' from 'utils.helpers' (/Users/robertliptak/Documents/Projects/chat_agent/utils/helpers.py)